# Goal
This script will transform raw target data (i.e., monthly inventory by state) and associated features into one DataFrame to be utilized for machine learning.

## 0) Setup

In [1]:
# Import dependencies.
import pandas as pd
import numpy as np
import sqlalchemy as sa
import psycopg2

## 1) Pull Target CSV (Inventory Data)
- Import raw Inventory CSV.
- Select, clean up, and format data.
- Redefine target data to be categorical (i.e., "Hot" or "Not Hot").

In [2]:
# Load in raw inventory CSV.
path = 'data/Target.csv'
target_df = pd.read_csv(path)
target_df.head()

,month_date_yyyymm,state,state_id,median_listing_price,median_listing_price_mm,median_listing_price_yy,active_listing_count,active_listing_count_mm,active_listing_count_yy,median_days_on_market,...,average_listing_price,average_listing_price_mm,average_listing_price_yy,total_listing_count,total_listing_count_mm,total_listing_count_yy,pending_ratio,pending_ratio_mm,pending_ratio_yy,quality_flag
0,202301,Alabama,AL,313495,0.0116,0.1654,11755,-0.0610,0.6740,78,...,410280,0.0131,0.0858,15873,-0.0412,0.2687,0.3513,0.0799,-0.4278,0.0
1,202212,Alabama,AL,309900,-0.0038,0.1442,12519,-0.0298,0.5652,69,...,404967,-0.0070,0.0719,16555,-0.0592,0.1882,0.3253,-0.1102,-0.4214,0.0
2,202211,Alabama,AL,311083,-0.0124,0.1312,12904,0.0314,0.5070,59,...,407813,-0.0184,0.0671,17597,-0.0107,0.1501,0.3656,-0.1551,-0.4229,0.0
3,202210,Alabama,AL,315000,-0.0139,0.1455,12511,0.0476,0.3801,54,...,415469,-0.0047,0.0938,17788,-0.0065,0.1185,0.4327,-0.1362,-0.3233,0.0
4,202209,Alabama,AL,319450,-0.0061,0.1555,11943,0.0111,0.2965,52,...,417431,-0.0082,0.0982,17905,0.0184,0.0892,0.5009,0.0195,-0.2863,0.0


In [3]:
# Drop unneeded features (e.g., yy columns).
df = target_df.drop(
    ['state', 'median_listing_price_yy', 'active_listing_count_yy', 'median_days_on_market_yy',\
        'new_listing_count_yy', 'price_increased_count_yy', 'price_reduced_count_yy',\
        'pending_listing_count', 'pending_listing_count_mm', 'pending_listing_count_yy', 'median_listing_price_per_square_foot_yy',\
        'median_square_feet_yy', 'average_listing_price_yy', 'total_listing_count_yy',\
        'pending_ratio', 'pending_ratio_mm', 'pending_ratio_yy', 'quality_flag'], axis=1
)

# Filter out rows for July 2016, given that it is the first month/year in the dataset and month-by-month features in this month/year will have errors.
df = df.loc[df['month_date_yyyymm'] != 201607]

In [4]:
# New column for categorical hot or not.
df['Target'] = np.where(df['total_listing_count_mm'] > 0, 'Hot', 'Not Hot' )
df.drop(['total_listing_count_mm'], axis=1, inplace=True)

# Display DataFrame.
print(len(df))
df.head()

3978


,month_date_yyyymm,state_id,median_listing_price,median_listing_price_mm,active_listing_count,active_listing_count_mm,median_days_on_market,median_days_on_market_mm,new_listing_count,new_listing_count_mm,...,price_reduced_count,price_reduced_count_mm,median_listing_price_per_square_foot,median_listing_price_per_square_foot_mm,median_square_feet,median_square_feet_mm,average_listing_price,average_listing_price_mm,total_listing_count,Target
0,202301,AL,313495,0.0116,11755,-0.0610,78,0.1304,5746,0.3501,...,3546,0.2866,154,0.0132,1966,-0.0020,410280,0.0131,15873,Not Hot
1,202212,AL,309900,-0.0038,12519,-0.0298,69,0.1695,4256,-0.2370,...,2756,-0.3485,152,0.0066,1970,-0.0115,404967,-0.0070,16555,Not Hot
2,202211,AL,311083,-0.0124,12904,0.0314,59,0.0926,5578,-0.1273,...,4230,-0.0482,151,-0.0066,1993,0.0010,407813,-0.0184,17597,Not Hot
3,202210,AL,315000,-0.0139,12511,0.0476,54,0.0385,6392,-0.2228,...,4444,0.0914,152,-0.0130,1991,0.0015,415469,-0.0047,17788,Not Hot
4,202209,AL,319450,-0.0061,11943,0.0111,52,0.1556,8224,0.1422,...,4072,-0.0355,154,0.0065,1988,-0.0020,417431,-0.0082,17905,Hot


In [5]:
df.dtypes

month_date_yyyymm                            int64
state_id                                    object
median_listing_price                         int64
median_listing_price_mm                    float64
active_listing_count                         int64
active_listing_count_mm                    float64
median_days_on_market                        int64
median_days_on_market_mm                   float64
new_listing_count                            int64
new_listing_count_mm                       float64
price_increased_count                        int64
price_increased_count_mm                   float64
price_reduced_count                          int64
price_reduced_count_mm                     float64
median_listing_price_per_square_foot         int64
median_listing_price_per_square_foot_mm    float64
median_square_feet                           int64
median_square_feet_mm                      float64
average_listing_price                        int64
average_listing_price_mm       

## 2) Pull Features from CSVs
Perform the following for each CSV:
- Load in raw CSV.
- Unpivot to match the DataFrame made from Section 1.
- Clean up data.
- Aggregate data by month and state, if necessary.
- Join to DataFrame by month and state.

### "Hotness" Feature

In [6]:
# Load in Hotness CSV.
hot_df = pd.read_csv('data/Hotness.csv')
hot_df.head()

C:\Users\pbhan\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (0,1) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,month_date_yyyymm,county_fips,county_name,cbsa_code,cbsa_title,nielsen_hh_rank,hotness_rank,hotness_rank_mm,hotness_rank_yy,hotness_score,...,median_dom_yy_day,median_dom_vs_us,ldp_unique_viewers_per_property_mm,ldp_unique_viewers_per_property_yy,ldp_unique_viewers_per_property_vs_us,median_listing_price,median_listing_price_mm,median_listing_price_yy,median_listing_price_vs_us,quality_flag
0,202302,39045,"fairfield, oh",18140.0,"Columbus, OH",432.0,1.0,-3.0,-15.0,99.529486,...,6.25,-36.75,0.129849,-0.282997,3.691870,369922.00,0.032998,0.181134,0.891486,0.0
1,202302,36055,"monroe, ny",40380.0,"Rochester, NY",76.0,2.0,1.0,0.0,99.466750,...,8.00,-48.00,0.090604,-0.368734,2.905487,197425.00,0.039626,-0.018762,0.475780,0.0
2,202302,9003,"hartford, ct",25540.0,"Hartford-East Hartford-Middletown, CT",56.0,3.0,-3.0,-34.0,99.341280,...,2.25,-35.75,0.080536,-0.154846,3.574108,361199.75,0.047108,0.070380,0.870466,0.0
3,202302,25009,"essex, ma",14460.0,"Boston-Cambridge-Newton, MA-NH",79.0,4.0,1.0,-3.0,99.153074,...,10.00,-36.00,0.090211,-0.411597,3.022914,697000.00,0.052234,0.064250,1.679720,0.0
4,202302,36117,"wayne, ny",40380.0,"Rochester, NY",635.0,5.0,-4.0,-145.0,98.368883,...,-6.25,-35.50,0.234980,-0.202389,2.591071,173650.00,-0.067150,-0.054451,0.418484,0.0


In [7]:
# Drop last row (not data)
lastrow = len(hot_df) - 1
hot_df.drop(lastrow, inplace=True)

In [8]:
hot_df.dtypes

month_date_yyyymm                         object
county_fips                               object
county_name                               object
cbsa_code                                float64
cbsa_title                                object
nielsen_hh_rank                          float64
hotness_rank                             float64
hotness_rank_mm                          float64
hotness_rank_yy                          float64
hotness_score                            float64
supply_score                             float64
demand_score                             float64
median_days_on_market                    float64
median_days_on_market_mm                 float64
median_dom_mm_day                        float64
median_days_on_market_yy                 float64
median_dom_yy_day                        float64
median_dom_vs_us                         float64
ldp_unique_viewers_per_property_mm       float64
ldp_unique_viewers_per_property_yy       float64
ldp_unique_viewers_p

In [9]:
# Change key to int for joining
hot_df['month_date_yyyymm'] = hot_df['month_date_yyyymm'].astype('int')
# Split state id for joining
hot_df[['county','state_id']] = hot_df['county_name'].str.split(', ',expand=True)
# state_id to capital letters
hot_df['state_id'] = hot_df['state_id'].str.upper()
# Reduce df
hotter_df = hot_df[['month_date_yyyymm', 'hotness_rank_mm', 'hotness_score', 'state_id']]
# Remove N/As (i.e., August 2017)
hotter_df = hotter_df.dropna()
hotter_df

,month_date_yyyymm,hotness_rank_mm,hotness_score,state_id
0,202302,-3.0,99.529486,OH
1,202302,1.0,99.466750,NY
2,202302,-3.0,99.341280,CT
3,202302,1.0,99.153074,MA
4,202302,-4.0,98.368883,NY
...,...,...,...,...
105265,201709,2.0,1.317440,SC
105266,201709,5.0,1.097867,CO
105267,201709,0.0,0.627353,AL
105268,201709,-1.0,0.533250,AR


In [10]:
hotter_gb_df = hotter_df.groupby(by=['state_id', 'month_date_yyyymm'])

In [11]:
hottest_df = hotter_gb_df.mean().reset_index()
hottest_df = hottest_df.rename(columns={"hotness_rank_mm": "avg_hotness_rank_mm", "hotness_score": "avg_hotness_score"})
hottest_df

,state_id,month_date_yyyymm,avg_hotness_rank_mm,avg_hotness_score
0,AK,201709,68.200,42.609787
1,AK,201710,30.200,40.846926
2,AK,201711,57.600,37.691343
3,AK,201712,-15.800,38.563363
4,AK,201801,-104.600,44.259724
...,...,...,...,...
3361,WY,202210,35.500,62.515684
3362,WY,202211,62.875,59.280897
3363,WY,202212,14.000,58.896644
3364,WY,202301,22.125,57.732120


### "Temperature (F)" Feature

In [12]:
# Load in Temperature CSV.
temp_df = pd.read_csv('data/US_Temp_AVG_Per_Month.csv')
temp_df

,State,Abbreviation,201607,201608,201609,201610,201611,201612,201701,201702,...,202204,202205,202206,202207,202208,202209,202210,202211,202212,202301
0,Alabama,AL,82.0,81.6,78.6,68.5,57.8,50.1,52.4,55.6,...,62.5,73.5,80.2,81.6,79.3,73.8,61.2,55.3,49.0,51.4
1,Alaska,AK,55.6,53.2,42.3,29.7,13.3,6.2,4.5,6.6,...,25.1,39.9,52.7,53.4,50.1,42.9,28.6,16.4,7.4,10.9
2,Arizona,AZ,82.4,77.5,71.6,66.1,53.1,44.7,42.3,49.4,...,60.8,68.6,78.5,82.0,78.6,75.5,61.6,46.0,43.0,40.5
3,Arkansas,AR,82.0,80.0,75.8,67.0,55.7,42.5,45.2,52.5,...,59.8,70.9,78.1,84.1,79.4,73.8,61.2,49.8,43.2,46.6
4,California,CA,76.8,76.2,69.5,60.5,52.8,43.4,42.2,47.3,...,55.8,62.3,72.0,77.6,78.3,73.5,64.1,46.6,43.1,42.7
5,Colorado,CO,69.2,64.5,59.7,51.8,40.2,25.2,25.3,35.4,...,43.7,53.3,64.3,70.3,67.8,62.5,47.6,31.3,25.7,24.2
6,Connecticut,CT,74.1,73.9,66.2,53.5,43.5,32.6,32.7,34.6,...,47.5,60.7,66.2,74.1,74.4,63.5,52.3,45.0,33.4,37.1
7,Delaware,DE,79.6,78.6,72.0,60.3,49.0,39.2,39.1,44.4,...,53.2,64.8,72.4,78.7,78.0,69.1,55.6,49.8,37.7,43.5
8,Florida,FL,84.0,83.0,81.1,74.6,66.6,65.9,62.8,65.5,...,71.8,77.6,81.9,83.0,82.8,79.9,71.8,69.7,61.4,62.1
9,Georgia,GA,82.8,81.6,77.5,67.8,58.0,52.1,53.2,56.5,...,63.1,73.3,80.1,81.2,79.5,74.0,62.1,57.6,49.1,51.7


In [13]:
# Drop unneeded State column
temp_df = temp_df.drop(['State'], axis=1)

# Melt date columns to match format
temp_df = temp_df.melt(
    id_vars=['Abbreviation'], var_name='month_date_yyyymm', value_name='temperature_F')

# Change state column to match other DF
temp_df = temp_df.rename(columns={'Abbreviation': 'state_id'})

# Change date type to int
temp_df['month_date_yyyymm'] = temp_df['month_date_yyyymm'].astype('int')

In [14]:
temp_df.dtypes

state_id              object
month_date_yyyymm      int32
temperature_F        float64
dtype: object

In [15]:
temp_df = temp_df.sort_values(by=['state_id', 'month_date_yyyymm'], ascending=False)
temp_df['temp_change_pct'] = temp_df['temperature_F'].pct_change(-1)
temp_df['temp_change_pct'] = temp_df['temp_change_pct'].fillna(0)
temp_df = temp_df.loc[temp_df['month_date_yyyymm'] != 201607]
temp_df

,state_id,month_date_yyyymm,temperature_F,temp_change_pct
3949,WY,202301,19.3,0.005208
3899,WY,202212,19.2,-0.222672
3849,WY,202211,24.7,-0.446188
3799,WY,202210,44.6,-0.254181
3749,WY,202209,59.8,-0.111441
...,...,...,...,...
251,AK,201612,6.2,-0.533835
201,AK,201611,13.3,-0.552189
151,AK,201610,29.7,-0.297872
101,AK,201609,42.3,-0.204887


### Unemployment Data

In [16]:
# Load in Unemployemnt CSV.
unemp_df = pd.read_csv('data/US_States_Unemployment_Rate_by_Month.csv')
unemp_df

,State,Abbreviation,201607,201608,201609,201610,201611,201612,201701,201702,...,202203,202204,202205,202206,202207,202208,202209,202210,202211,202212
0,Alabama,AL,5.9,5.9,5.9,5.9,5.8,5.7,5.5,5.2,...,2.9,2.8,2.7,2.6,2.6,2.6,2.6,2.7,2.7,2.8
1,Alaska,AK,6.6,6.7,6.7,6.7,6.7,6.7,6.6,6.6,...,4.9,4.8,4.6,4.6,4.5,4.6,4.4,4.5,4.5,4.3
2,Arizona,AZ,5.5,5.4,5.4,5.4,5.3,5.3,5.2,5.2,...,3.3,3.2,3.2,3.3,3.3,3.5,3.7,3.9,4.1,4.0
3,Arkansas,AR,4.0,4.0,3.9,3.9,3.9,3.8,3.8,3.7,...,3.1,3.2,3.2,3.2,3.3,3.4,3.5,3.6,3.7,3.6
4,California,CA,5.5,5.5,5.5,5.5,5.4,5.4,5.2,5.1,...,4.8,4.6,4.3,4.2,3.9,4.1,3.8,4.0,4.1,4.1
5,Colorado,CO,3.2,3.1,3.0,3.0,2.9,2.7,2.6,2.5,...,3.7,3.6,3.5,3.4,3.3,3.4,3.4,3.6,3.5,3.3
6,Connecticut,CT,4.8,4.7,4.7,4.7,4.6,4.6,4.6,4.5,...,3.7,3.6,3.5,3.4,3.3,3.4,3.4,3.6,3.5,3.3
7,Delaware,DE,4.5,4.5,4.6,4.6,4.7,4.7,4.7,4.6,...,4.5,4.5,4.5,4.5,4.4,4.5,4.3,4.3,4.4,4.4
8,Florida,FL,4.9,4.9,4.9,4.9,4.8,4.8,4.7,4.6,...,3.2,3.0,2.9,2.8,2.7,2.7,2.5,2.7,2.6,2.5
9,Georgia,GA,5.4,5.4,5.4,5.4,5.4,5.3,5.2,5.1,...,3.1,3.1,3.0,2.9,2.8,2.8,2.8,2.9,3.0,3.0


In [17]:
# Drop unneeded State column
unemp_df = unemp_df.drop(['State'], axis=1)

# Melt date columns to match format
unemp_df = unemp_df.melt(
    id_vars=['Abbreviation'], var_name='month_date_yyyymm', value_name='unemployment_rate')

# Change state column to match other DF
unemp_df = unemp_df.rename(columns={'Abbreviation': 'state_id'})

# Change date type to int
unemp_df['month_date_yyyymm'] = unemp_df['month_date_yyyymm'].astype('int')

In [18]:
unemp_df.dtypes

state_id              object
month_date_yyyymm      int32
unemployment_rate    float64
dtype: object

In [20]:
unemp_df = unemp_df.sort_values(by=['state_id', 'month_date_yyyymm'], ascending=False)
unemp_df['unemployment_rate_change_pct'] = unemp_df['unemployment_rate'].pct_change(-1)
unemp_df['unemployment_rate_change_pct'] = unemp_df['unemployment_rate_change_pct'].fillna(0)
unemp_df = unemp_df.loc[unemp_df['month_date_yyyymm'] != 201607]
unemp_df

,state_id,month_date_yyyymm,unemployment_rate,unemployment_rate_change_pct
3899,WY,202212,3.7,0.027778
3849,WY,202211,3.6,0.028571
3799,WY,202210,3.5,0.060606
3749,WY,202209,3.3,0.064516
3699,WY,202208,3.1,0.033333
...,...,...,...,...
251,AK,201612,6.7,0.000000
201,AK,201611,6.7,0.000000
151,AK,201610,6.7,0.000000
101,AK,201609,6.7,0.000000


## 3) Merge DataFrames

In [24]:
final_df = df.merge(hottest_df, on=['month_date_yyyymm', 'state_id'], how='inner')
final_df = final_df.merge(temp_df, on=['month_date_yyyymm', 'state_id'], how='inner')
final_df = final_df.merge(unemp_df, on=['month_date_yyyymm', 'state_id'], how='inner')

final_df['season'] = ['Spring' if x in ['03', '04', '05'] else 'Summer' if x in ['06', '07', '08'] else 'Fall' if x in ['09', '10', '11'] else 'Winter' if x in ['12', '01', '02'] else 'N/A' for x in final_df['month_date_yyyymm'].astype('str').str[-2:]]

final_df = final_df.drop(columns=['month_date_yyyymm', 'state_id'])
final_df

,median_listing_price,median_listing_price_mm,active_listing_count,active_listing_count_mm,median_days_on_market,median_days_on_market_mm,new_listing_count,new_listing_count_mm,price_increased_count,price_increased_count_mm,...,average_listing_price_mm,total_listing_count,Target,avg_hotness_rank_mm,avg_hotness_score,temperature_F,temp_change_pct,unemployment_rate,unemployment_rate_change_pct,season
0,309900,-0.0038,12519,-0.0298,69,0.1695,4256,-0.2370,240,-0.3296,...,-0.0070,16555,Not Hot,0.048780,36.241087,49.0,-0.113924,2.8,0.037037,Winter
1,311083,-0.0124,12904,0.0314,59,0.0926,5578,-0.1273,358,0.1623,...,-0.0184,17597,Not Hot,36.048780,36.361202,55.3,-0.096405,2.7,0.000000,Fall
2,315000,-0.0139,12511,0.0476,54,0.0385,6392,-0.2228,308,-0.1047,...,-0.0047,17788,Not Hot,-6.731707,37.750558,61.2,-0.170732,2.7,0.038462,Fall
3,319450,-0.0061,11943,0.0111,52,0.1556,8224,0.1422,344,0.1862,...,-0.0082,17905,Hot,64.902439,37.749793,73.8,-0.069357,2.6,0.000000,Fall
4,321420,-0.0028,11812,0.0505,45,0.1842,7200,-0.1018,290,-0.0823,...,-0.0058,17581,Hot,-30.804878,40.812345,79.3,-0.028186,2.6,0.000000,Summer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3195,244900,0.0082,3453,-0.0733,131,0.0650,912,0.5616,18,0.1250,...,0.0075,3822,Not Hot,-0.875000,41.428795,25.1,0.054622,4.2,-0.023256,Winter
3196,242900,0.0048,3726,-0.0814,123,0.1284,584,-0.2255,16,0.0000,...,0.0070,4042,Not Hot,18.250000,41.373902,23.8,-0.314121,4.3,0.000000,Winter
3197,241750,-0.0246,4056,-0.1007,109,0.0792,754,-0.0981,16,-0.1111,...,-0.0539,4397,Not Hot,21.250000,42.405113,34.7,-0.155718,4.3,0.000000,Fall
3198,247850,-0.0002,4510,-0.0493,101,0.1099,836,-0.1992,18,-0.4375,...,-0.0021,4873,Not Hot,49.500000,43.585320,41.1,-0.241697,4.3,0.000000,Fall


## 4) Load to SQL Database
Before this, set up new database & table in Postgres.
- Remove rows where there are any empty features (?).
- Check datatypes and cast to proper datatype (i.e., one that makes sense and one that matches the SQL table).

In [ ]:
# final_df.to_sql(name='hotness', con=conn, if_exists='append')